Da es kaum möglich ist zu raten, welche featues einen einfluss auf den Umsatz von Croissants haben, versuche ich diese über Korrelationen zu bestimmen.

In [2]:
import pandas as pd
import numpy as np

#read in the data
data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")

#drop all rows that are not warengruppe 3
data = data[data["Warengruppe_3"] == 1]

#drop all columns that I dont want to calculate the correlation for
data = data.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6", 'Datum'])
data.head()

,Umsatz,Temp_Very_Cold,Temp_Cold,Temp_Mild,Temp_Warm,Temp_Hot,Cloud_Clear,Cloud_Partly_Cloudy,Cloud_Cloudy,Wind_Light,...,Monat_7,Monat_8,Monat_9,Monat_10,Monat_11,Monat_12,zwischen_den_jahren,Number_of_ships_scaled,Cloud_ok,Wind_ok
3638,201.198426,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0.0,0,1
3639,265.261254,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0.0,1,1
3640,210.260241,0,0,0,1,0,0,0,1,1,...,1,0,0,0,0,0,0,0.0,0,1
3641,190.686641,0,0,1,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0.0,0,1
3642,181.644870,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0.2,1,1


In [4]:
#lineare correlation zwischen den variablen und der zielvariable berechnen
correlation = data.corr()["Umsatz"].sort_values(ascending=False)
#correlation = correlation.drop("Umsatz")
correlation


Umsatz                    1.000000
Sommer                    0.632812
Monat_8                   0.467922
Monat_7                   0.430227
is_holiday                0.429896
Temp_Warm                 0.377216
wetter_sehr_schön         0.285412
Number_of_ships_scaled    0.269484
Number_of_ships           0.269484
Ship                      0.264904
Sonntag                   0.241471
Temp_Hot                  0.230605
Cloud_ok                  0.228773
Samstag                   0.206775
Temp_Mild                 0.184706
Cloud_Clear               0.147291
KielerWoche               0.126438
Cloud_Partly_Cloudy       0.118924
Feiertag                  0.116342
Ostertag                  0.096434
Wind_Moderate             0.079509
Monat_6                   0.076073
Heimspiel                 0.063103
VPI                       0.059597
Wind_ok                   0.043476
Monat_10                  0.022659
Weather_Good              0.021357
Silvester                 0.011746
Weather_Light_Issues

In [5]:
#nicht lineare correlation berechnen
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

X = data.drop(columns=["Umsatz"])
y = data["Umsatz"]
mi_scores = make_mi_scores(X, y)
mi_scores

VPI                       0.256117
Sommer                    0.224981
Temp_Cold                 0.156736
Winter                    0.128611
Monat_8                   0.103448
Werktag                   0.093452
Monat_7                   0.091562
Temp_Warm                 0.089850
is_holiday                0.088017
Temp_Mild                 0.069667
Sonntag                   0.062396
Weihnachtsmarkt           0.044659
wetter_sehr_schön         0.038849
Number_of_ships           0.037628
Number_of_ships_scaled    0.037578
Monat_1                   0.034566
Monat_12                  0.027470
Herbst                    0.026533
Cloud_ok                  0.025777
Cloud_Cloudy              0.025729
Monat_9                   0.025499
Ship                      0.025205
Cloud_Clear               0.024482
Samstag                   0.024440
Monat_3                   0.024324
Monat_10                  0.023072
Temp_Hot                  0.022956
Monat_2                   0.022818
Monat_6             

In [6]:
# Anzahl der Top-Features, die ausgewählt werden sollen
n = 20

# Top-n Features basierend auf Korrelation
top_n_correlation = correlation.index[:n].tolist()

# Top-n Features basierend auf Mutual Information Scores
top_n_mi_scores = mi_scores.index[:n].tolist()

# Kombinieren der ausgewählten Features
selected_features = list(set(top_n_correlation + top_n_mi_scores))

print("Ausgewählte Features:", selected_features)

Ausgewählte Features: ['Temp_Mild', 'Sommer', 'Cloud_Clear', 'Umsatz', 'Temp_Cold', 'VPI', 'Monat_1', 'Temp_Hot', 'Cloud_ok', 'Ostertag', 'Samstag', 'Number_of_ships_scaled', 'Monat_12', 'Temp_Warm', 'KielerWoche', 'Monat_8', 'Feiertag', 'Winter', 'Herbst', 'Weihnachtsmarkt', 'is_holiday', 'Number_of_ships', 'Werktag', 'Sonntag', 'Monat_7', 'wetter_sehr_schön', 'Cloud_Cloudy', 'Cloud_Partly_Cloudy', 'Ship']
